In [2]:
#Task 1: How many users, activities and trackpoints are there in the dataset (after it isinserted into the database)

from DbConnector import DbConnector


db_connector = DbConnector(DATABASE='strava_mongoDB', HOST="localhost", USER="admin", PASSWORD="admin123")
db = db_connector.db


users_collection = db['users']
activities_collection = db['activities']


user_count = users_collection.count_documents({})
print(f"Total number of users: {user_count}")


activity_count = activities_collection.count_documents({})
print(f"Total number of activities: {activity_count}")


pipeline = [
    {"$unwind": "$trackpoints"}, 
    {"$group": {"_id": None, "totalTrackpoints": {"$sum": 1}}}  
]

trackpoint_count_result = list(activities_collection.aggregate(pipeline))


trackpoint_count = trackpoint_count_result[0]['totalTrackpoints'] if trackpoint_count_result else 0
print(f"Total number of trackpoints: {trackpoint_count}")

db_connector.close_connection()


You are connected to the database: strava_mongoDB
-----------------------------------------------

Total number of users: 182
Total number of activities: 16048
Total number of trackpoints: 9681756

-----------------------------------------------
Connection to strava_mongoDB-db is closed


In [3]:
#Task 2: Find the average number of activities per user

from DbConnector import DbConnector


db_connector = DbConnector(DATABASE='strava_mongoDB', HOST="localhost", USER="admin", PASSWORD="admin123")
db = db_connector.db


users_collection = db['users']


pipeline = [
    {
        "$project": {
            "num_activities": {"$size": "$activity_ids"}  
        }
    },
    {
        "$group": {
            "_id": None,
            "total_activities": {"$sum": "$num_activities"},  
            "total_users": {"$sum": 1}  
        }
    }
]


result = list(users_collection.aggregate(pipeline))


if result and result[0]['total_users'] > 0:
    total_activities = result[0]['total_activities']
    total_users = result[0]['total_users']
    average_activities_per_user = total_activities / total_users
    print(f"Average number of activities per user: {average_activities_per_user:.2f}")
else:
    print("No users found or no activities data.")


db_connector.close_connection()



You are connected to the database: strava_mongoDB
-----------------------------------------------

Average number of activities per user: 88.18

-----------------------------------------------
Connection to strava_mongoDB-db is closed


In [4]:
#Task 3: 
# Find the top 20 users with the highest number of activities. 


from DbConnector import DbConnector


db_connector = DbConnector(DATABASE='strava_mongoDB', HOST="localhost", USER="admin", PASSWORD="admin123")


db = db_connector.db
users_collection = db['users']
activities_collection = db['activities']


pipeline = [
    {
        "$project": {
            "_id": 1,  
            "num_activities": { "$size": "$activity_ids" }  
        }
    },
    {
        "$sort": { "num_activities": -1 }  
    },
    {
        "$limit": 20  
    }
]


top_users = list(users_collection.aggregate(pipeline))


for user in top_users:
    print(f"User ID: {user['_id']}, Number of Activities: {user['num_activities']}")





You are connected to the database: strava_mongoDB
-----------------------------------------------

User ID: 128, Number of Activities: 2102
User ID: 153, Number of Activities: 1793
User ID: 25, Number of Activities: 715
User ID: 163, Number of Activities: 704
User ID: 62, Number of Activities: 691
User ID: 144, Number of Activities: 563
User ID: 41, Number of Activities: 399
User ID: 85, Number of Activities: 364
User ID: 4, Number of Activities: 346
User ID: 140, Number of Activities: 345
User ID: 167, Number of Activities: 320
User ID: 68, Number of Activities: 280
User ID: 17, Number of Activities: 265
User ID: 3, Number of Activities: 261
User ID: 14, Number of Activities: 236
User ID: 126, Number of Activities: 215
User ID: 30, Number of Activities: 210
User ID: 112, Number of Activities: 208
User ID: 11, Number of Activities: 201
User ID: 39, Number of Activities: 198


In [5]:
# Task 4: 
# Find all users who have taken a taxi




taxi_activities = activities_collection.find({"transportation_mode": "taxi"}, {"_id": 1})


taxi_activity_ids = [activity["_id"] for activity in taxi_activities]


taxi_users = users_collection.find({"activity_ids": {"$in": taxi_activity_ids}})


print("Users who have taken a taxi:")
for user in taxi_users:
    print(f"User ID: {user['_id']}")


Users who have taken a taxi:
User ID: 10
User ID: 21
User ID: 52
User ID: 56
User ID: 58
User ID: 62
User ID: 65
User ID: 78
User ID: 80
User ID: 84
User ID: 85
User ID: 98
User ID: 111
User ID: 114
User ID: 126
User ID: 128
User ID: 139
User ID: 153
User ID: 163
User ID: 167
User ID: 175


In [6]:
# Task 5:

# Find all types of transportation modes and count how many activities that are
# tagged with these transportation mode labels. Do not count the rows where
# the mode is null.



pipeline = [
    {
        
        "$match": {"transportation_mode": {"$ne": None}}
    },
    {
        
        "$group": {
            "_id": "$transportation_mode",  
            "count": {"$sum": 1}  
        }
    },
    {
        
        "$sort": {"count": -1}
    }
]


transportation_modes = list(activities_collection.aggregate(pipeline))


print("Transportation modes and their activity counts:")
for mode in transportation_modes:
    print(f"Mode: {mode['_id']}, Count: {mode['count']}")




Transportation modes and their activity counts:
Mode: walk, Count: 1008
Mode: bike, Count: 619
Mode: car, Count: 493
Mode: bus, Count: 471
Mode: subway, Count: 190
Mode: taxi, Count: 125
Mode: airplane, Count: 4
Mode: train, Count: 2
Mode: boat, Count: 1
Mode: run, Count: 1


In [7]:
# 6a
# Find the year with the most activities


pipeline = [
    {
        "$group": {
            "_id": {"$year": "$start_date_time"},  
            "count": {"$sum": 1}  
        }
    },
    {
        "$sort": {"count": -1}  
    },
    {
        "$limit": 1  
    }
]


result = list(activities_collection.aggregate(pipeline))


if result:
    most_active_year = result[0]['_id']
    activity_count = result[0]['count']
    print(f"The year with the most activities is {most_active_year} with {activity_count} activities.")
else:
    print("No activities found.")




The year with the most activities is 2008 with 5895 activities.


In [8]:
# 6b


pipeline = [
    {
        "$addFields": {
            "duration_hours": {
                "$divide": [
                    {"$subtract": ["$end_date_time", "$start_date_time"]},
                    1000 * 60 * 60  
                ]
            }
        }
    },
    {
        "$group": {
            "_id": {"$year": "$start_date_time"},  
            "total_hours": {"$sum": "$duration_hours"},  
            "activity_count": {"$sum": 1}  
        }
    },
    {
        "$sort": {"total_hours": -1}  
    },
    {
        "$limit": 1  
    }
]


result = list(activities_collection.aggregate(pipeline))


if result:
    year_with_most_hours = result[0]['_id']
    total_hours = result[0]['total_hours']
    activity_count_in_most_hours_year = result[0]['activity_count']
    print(f"The year with the most recorded hours is {year_with_most_hours} with {total_hours} hours.")
    
    
    if most_active_year == year_with_most_hours:
        print("Yes, this is also the year with the most activities.")
    else:
        print("No, this is a different year than the year with the most activities.")
else:
    print("No activities found.")




The year with the most recorded hours is 2009 with 11612.423888888889 hours.
No, this is a different year than the year with the most activities.


In [9]:
# 7
# Find the total distance (in km) walked in 2008, by user with id=112.


user = users_collection.find_one({"_id": "112"})


if user and "activity_ids" in user:
    activity_ids = user["activity_ids"]
   
    pipeline = [
        {
            "$match": {
                "_id": {"$in": activity_ids},  
                "transportation_mode": "walk",
                "$expr": { "$eq": [{ "$year": "$start_date_time" }, 2008] }  
            }
        },
        {
            "$group": {
                "_id": None,
                "total_distance_walked_km": {"$sum": "$total_distance_km"}  
            }
        }
    ]
    
    
    result = list(activities_collection.aggregate(pipeline))
    
  
    if result:
        total_distance_walked = result[0]['total_distance_walked_km']
        print(f"The total distance walked by user 112 in 2008 is {total_distance_walked} km.")
    else:
        print("No walking activities found for user 112 in 2008.")
else:
    print("User 112 does not exist or has no associated activities.")



The total distance walked by user 112 in 2008 is 256.3828245787455 km.


In [10]:
# 8
# Find the top 20 users who have gained the most altitude meters.

pipeline = [
    {"$unwind": "$trackpoints"},
    {"$sort": {"_id": 1, "trackpoints.date_time": 1}},

   
    {
        "$group": {
            "_id": "$_id",
            "altitudes": {"$push": "$trackpoints.altitude"}
        }
    },

   
    {
        "$addFields": {
            "altitude_gain": {
                "$reduce": {
                    "input": {"$range": [1, {"$size": "$altitudes"}]},
                    "initialValue": 0,
                    "in": {
                        "$add": [
                            "$$value",
                            {
                                "$cond": [
                                    {"$gt": [
                                        {"$subtract": [
                                            {"$arrayElemAt": ["$altitudes", "$$this"]},
                                            {"$arrayElemAt": ["$altitudes", {"$subtract": ["$$this", 1]}]}
                                        ]},
                                        0
                                    ]},
                                    {
                                        "$multiply": [
                                            {"$subtract": [
                                                {"$arrayElemAt": ["$altitudes", "$$this"]},
                                                {"$arrayElemAt": ["$altitudes", {"$subtract": ["$$this", 1]}]}
                                            ]},
                                            0.3048  # Feet to meters conversion
                                        ]
                                    },
                                    0
                                ]
                            }
                        ]
                    }
                }
            }
        }
    },

  
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "activity_ids",
            "as": "user"
        }
    },
    {"$unwind": "$user"},
    {
        "$group": {
            "_id": "$user._id",
            "total_altitude_gain": {"$sum": "$altitude_gain"}
        }
    },

   
    {"$sort": {"total_altitude_gain": -1}},
    {"$limit": 20},
    {"$project": {"_id": 1, "total_altitude_gain": 1}}
]


try:
    top_altitude_gainers = activities_collection.aggregate(pipeline)
    for doc in top_altitude_gainers:
        print(f"User ID: {doc['_id']}, Total Altitude Gain: {doc['total_altitude_gain']} meters")
except Exception as e:
    print("Error running aggregation pipeline:", e)




User ID: 128, Total Altitude Gain: 578423.4656800001 meters
User ID: 153, Total Altitude Gain: 512319.2969909993 meters
User ID: 4, Total Altitude Gain: 299814.6911999981 meters
User ID: 3, Total Altitude Gain: 209375.34959999876 meters
User ID: 163, Total Altitude Gain: 194675.4086199999 meters
User ID: 85, Total Altitude Gain: 194128.73543999958 meters
User ID: 144, Total Altitude Gain: 170648.13592 meters
User ID: 30, Total Altitude Gain: 155327.90879999896 meters
User ID: 39, Total Altitude Gain: 128172.97199999948 meters
User ID: 62, Total Altitude Gain: 125512.98240000001 meters
User ID: 41, Total Altitude Gain: 123154.22663999994 meters
User ID: 84, Total Altitude Gain: 118772.02559999954 meters
User ID: 167, Total Altitude Gain: 108435.37783999987 meters
User ID: 2, Total Altitude Gain: 106698.59279999946 meters
User ID: 0, Total Altitude Gain: 105817.1111999995 meters
User ID: 25, Total Altitude Gain: 85079.67744 meters
User ID: 126, Total Altitude Gain: 77769.98647999988 mete

In [11]:
# 9
# Find all users who have invalid activities, and the number of invalid activities per user 


user_invalid_activity_counts = []


for user in users_collection.find({}, {"_id": 1, "activity_ids": 1}):
    user_id = user["_id"]
    activity_ids = user.get("activity_ids", [])

    
    pipeline = [
        {
            "$match": {
                "_id": {"$in": activity_ids}  
            }
        },
        {
            "$project": {
                "trackpoints": "$trackpoints.date_time"
            }
        },
        {
            "$project": {
                "invalid_activity": {
                    "$anyElementTrue": {
                        "$map": {
                            "input": { "$range": [1, { "$size": "$trackpoints" }] },
                            "as": "idx",
                            "in": {
                                "$gte": [
                                    { "$subtract": [
                                        { "$arrayElemAt": ["$trackpoints", "$$idx"] },
                                        { "$arrayElemAt": ["$trackpoints", { "$subtract": ["$$idx", 1] }] }
                                    ] },
                                    5 * 60 * 1000  # 5 minutes in milliseconds
                                ]
                            }
                        }
                    }
                }
            }
        },
        {
            "$match": {
                "invalid_activity": True  
            }
        },
        {
            "$count": "invalid_activity_count"  
        }
    ]

   
    result = list(activities_collection.aggregate(pipeline))

    
    if result:
        invalid_count = result[0]["invalid_activity_count"]
        user_invalid_activity_counts.append({"user_id": user_id, "invalid_activity_count": invalid_count})


for user in user_invalid_activity_counts:
    print(f"User ID: {user['user_id']}, Invalid Activity Count: {user['invalid_activity_count']}")


User ID: 0, Invalid Activity Count: 101
User ID: 1, Invalid Activity Count: 45
User ID: 2, Invalid Activity Count: 98
User ID: 3, Invalid Activity Count: 179
User ID: 4, Invalid Activity Count: 219
User ID: 5, Invalid Activity Count: 45
User ID: 6, Invalid Activity Count: 17
User ID: 7, Invalid Activity Count: 30
User ID: 8, Invalid Activity Count: 16
User ID: 9, Invalid Activity Count: 31
User ID: 10, Invalid Activity Count: 50
User ID: 11, Invalid Activity Count: 32
User ID: 12, Invalid Activity Count: 43
User ID: 13, Invalid Activity Count: 29
User ID: 14, Invalid Activity Count: 118
User ID: 15, Invalid Activity Count: 46
User ID: 16, Invalid Activity Count: 20
User ID: 17, Invalid Activity Count: 129
User ID: 18, Invalid Activity Count: 27
User ID: 19, Invalid Activity Count: 31
User ID: 20, Invalid Activity Count: 20
User ID: 21, Invalid Activity Count: 7
User ID: 22, Invalid Activity Count: 55
User ID: 23, Invalid Activity Count: 11
User ID: 24, Invalid Activity Count: 27
User I

In [12]:
# 10 
# Find the users who have tracked an activity in the Forbidden City of Beijing. 


# Define the latitude and longitude range for Forbidden City (±0.001 tolerance)
lat_range = [39.916 - 0.001, 39.916 + 0.001]
lon_range = [116.397 - 0.001, 116.397 + 0.001]


pipeline = [
    {
        "$match": {
            "trackpoints": {
                "$elemMatch": {
                    "lat": { "$gte": lat_range[0], "$lte": lat_range[1] },
                    "lon": { "$gte": lon_range[0], "$lte": lon_range[1] }
                }
            }
        }
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "activity_ids",
            "as": "user_data"
        }
    },
    {
        "$unwind": "$user_data"  
    },
    {
        "$group": {
            "_id": "$user_data._id"  
        }
    }
]


result = list(activities_collection.aggregate(pipeline))


if result:
    print("Users who have tracked an activity near the Forbidden City:")
    for user in result:
        print(f"User ID: {user['_id']}")
else:
    print("No users have tracked activities near the Forbidden City.")




Users who have tracked an activity near the Forbidden City:
User ID: 19
User ID: 131
User ID: 18
User ID: 4


In [13]:
# 11.Find all users who have registered transportation_mode and their most used transportation_mode


pipeline = [
    
    {
        "$match": {
            "transportation_mode": {"$ne": None}
        }
    },
    
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",  
            "foreignField": "activity_ids",  
            "as": "user"
        }
    },
   
    {
        "$unwind": "$user"
    },
    
    {
        "$group": {
            "_id": {
                "user_id": "$user._id",
                "transportation_mode": "$transportation_mode"
            },
            "mode_count": {"$sum": 1}
        }
    },
    # Sort each user_id's transportation_mode count in descending order of usage count,
    # then by transportation_mode alphabetically to handle ties
    {
        "$sort": {
            "_id.user_id": 1,
            "mode_count": -1,
            "_id.transportation_mode": 1
        }
    },
    # Group by user_id to get the most used transportation mode (alphabetical in case of tie)
    {
        "$group": {
            "_id": "$_id.user_id",
            "most_used_transportation_mode": {"$first": "$_id.transportation_mode"}
        }
    },
    
    {
        "$project": {
            "_id": 1,
            "most_used_transportation_mode": 1
        }
    }
]


most_used_transportation_modes = activities_collection.aggregate(pipeline)

for doc in most_used_transportation_modes:
    print(f"User ID: {doc['_id']}, Most Used Transportation Mode: {doc['most_used_transportation_mode']}")




User ID: 52, Most Used Transportation Mode: bus
User ID: 167, Most Used Transportation Mode: bike
User ID: 101, Most Used Transportation Mode: car
User ID: 114, Most Used Transportation Mode: taxi
User ID: 60, Most Used Transportation Mode: walk
User ID: 138, Most Used Transportation Mode: bike
User ID: 141, Most Used Transportation Mode: walk
User ID: 89, Most Used Transportation Mode: car
User ID: 117, Most Used Transportation Mode: walk
User ID: 82, Most Used Transportation Mode: walk
User ID: 81, Most Used Transportation Mode: bike
User ID: 161, Most Used Transportation Mode: walk
User ID: 153, Most Used Transportation Mode: walk
User ID: 20, Most Used Transportation Mode: bike
User ID: 53, Most Used Transportation Mode: walk
User ID: 56, Most Used Transportation Mode: bike
User ID: 92, Most Used Transportation Mode: walk
User ID: 10, Most Used Transportation Mode: taxi
User ID: 144, Most Used Transportation Mode: walk
User ID: 102, Most Used Transportation Mode: walk
User ID: 97, 

In [17]:
from pprint import pprint 

# Show the first 10 documents in Activity Collection
first_10_users = users_collection.find().limit(10)

for user in first_10_users:
    pprint(user)

{'_id': '0',
 'activity_ids': ['1',
                  '2',
                  '3',
                  '4',
                  '5',
                  '6',
                  '7',
                  '8',
                  '9',
                  '10',
                  '11',
                  '12',
                  '13',
                  '14',
                  '15',
                  '16',
                  '17',
                  '18',
                  '19',
                  '20',
                  '21',
                  '22',
                  '23',
                  '24',
                  '25',
                  '26',
                  '27',
                  '28',
                  '29',
                  '30',
                  '31',
                  '32',
                  '33',
                  '34',
                  '35',
                  '36',
                  '37',
                  '38',
                  '39',
                  '40',
                  '41',
            

In [18]:
# Show the first 10 documents in Activity Collection
first_10_activities = activities_collection.find().limit(10)

for activity in first_10_activities:
    pprint(activity)

{'_id': '1',
 'end_date_time': datetime.datetime(2008, 10, 23, 11, 11, 12),
 'start_date_time': datetime.datetime(2008, 10, 23, 2, 53, 4),
 'total_distance_km': 14.938642126842497,
 'trackpoints': [{'_id': 1,
                  'altitude': 492.0,
                  'date_days': 39744.1201851852,
                  'date_time': datetime.datetime(2008, 10, 23, 2, 53, 4),
                  'lat': 39.984702,
                  'lon': 116.318417},
                 {'_id': 2,
                  'altitude': 492.0,
                  'date_days': 39744.1202546296,
                  'date_time': datetime.datetime(2008, 10, 23, 2, 53, 10),
                  'lat': 39.984683,
                  'lon': 116.31845},
                 {'_id': 3,
                  'altitude': 492.0,
                  'date_days': 39744.1203125,
                  'date_time': datetime.datetime(2008, 10, 23, 2, 53, 15),
                  'lat': 39.984686,
                  'lon': 116.318417},
                 {'_id': 4,
       